# Kshitij sonje CS24MTECh11025

# Assignment 3

In [1]:
import csv
import random
import numpy as np
from scipy.linalg import null_space

np.random.seed(0)
random.seed(0)

In [2]:
# Extract starting point and objective coefficients from CSV data
def extract_objective_data(raw_data):
    start_point = np.array([float(val) for val in raw_data[0][:-1]])
    cost_coeffs = np.array([float(val) for val in raw_data[1][:-1]])
    return start_point, cost_coeffs

# Extract constraint bounds from CSV data
def extract_constraint_bounds(raw_data):
    return np.array([float(row[-1]) for row in raw_data[2:]])

# Extract constraint coefficient matrix from CSV data
def extract_constraint_matrix(raw_data):
    return np.array([[float(val) for val in row[:-1]] for row in raw_data[2:]])

In [3]:
# Parse CSV file and return problem data
def parse_csv_input(filename, has_initial_point=True):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        raw_data = list(reader)
    
    start_point, cost_coeffs = extract_objective_data(raw_data)
    bounds = extract_constraint_bounds(raw_data)
    constraint_mat = extract_constraint_matrix(raw_data)
    
    return constraint_mat, start_point, bounds, cost_coeffs

In [4]:
# Compute active constraint mask based on tolerance
def compute_constraint_products(coeff_matrix, current_pt, bounds, tolerance=1e-8):
    constraint_vals = np.dot(coeff_matrix, current_pt)
    active_mask = np.abs(constraint_vals - bounds) < tolerance
    return active_mask

# Separate constraints into active and inactive sets
def separate_constraint_sets(coeff_matrix, active_mask):
    active_constraints = coeff_matrix[active_mask]
    inactive_constraints = coeff_matrix[~active_mask]
    return active_constraints, inactive_constraints

In [5]:
# Analyze constraints and return active/inactive sets
def analyze_constraints(coeff_matrix, current_pt, bounds, tolerance=1e-8):
    active_mask = compute_constraint_products(coeff_matrix, current_pt, bounds, tolerance)
    active_set, inactive_set = separate_constraint_sets(coeff_matrix, active_mask)
    return active_mask, active_set, inactive_set

In [6]:
# Invert matrix and negate for movement directions
def invert_and_negate_matrix(active_matrix):
    matrix_inverse = np.linalg.inv(active_matrix)
    negated_inverse = -1 * matrix_inverse
    return negated_inverse

# Handle singular matrix cases
def handle_singular_matrix():
    return None


In [7]:
# Compute movement direction vectors
def compute_movement_vectors(active_matrix):
    try:
        return invert_and_negate_matrix(active_matrix)
    except np.linalg.LinAlgError:
        return handle_singular_matrix()

In [8]:
# Check if constraint matrix is degenerate
def is_degenerate(active_constraint_matrix):
    if len(active_constraint_matrix) == 0:
        return False
    
    rows, cols = active_constraint_matrix.shape
    
    if rows > cols:
        return True
    try:
        rank = np.linalg.matrix_rank(active_constraint_matrix)
        if rank < min(rows, cols):
            return True
    except np.linalg.LinAlgError:
        return True
    return False

In [9]:
# Apply epsilon perturbation to constraint bounds
def apply_perturbation_terms(original_bounds, epsilon_val):
    adjusted_epsilon = epsilon_val
    perturbed_bounds = np.array([
        original_bounds[idx] + adjusted_epsilon**(idx+1) 
        for idx in range(len(original_bounds))
    ])
    return perturbed_bounds, adjusted_epsilon

# Remove degeneracy using perturbation technique
def remove_degeneracy_condition(original_constraint_bounds, epsilon_value, shrink_rate=0.5):
    reduced_epsilon = epsilon_value * shrink_rate
    modified_bounds, final_epsilon = apply_perturbation_terms(
        original_constraint_bounds, 
        reduced_epsilon
    )
    return modified_bounds, final_epsilon

In [10]:
# Build extended constraint matrix with auxiliary slack variable
def construct_augmented_constraint_matrix(coeff_matrix, total_rows, total_cols):
    expanded_matrix = np.append(coeff_matrix, np.zeros((1, total_cols)), axis=0)
    final_matrix = np.append(expanded_matrix, np.ones((total_rows + 1, 1)), axis=1)
    final_matrix[-1][-1] = -1
    return final_matrix

# Modify bounds vector and objective function for auxiliary problem
def adjust_bounds_and_objectives(bounds_vector, variable_count):
    extended_bounds = np.append(bounds_vector, [abs(min(bounds_vector))], axis=0)
    auxiliary_objective = np.zeros((variable_count + 1, 1))
    auxiliary_objective[-1] = 1
    return extended_bounds, auxiliary_objective

In [11]:
# Generate auxiliary linear program formulation for initial feasible solution
def create_phase_one_formulation(coeff_matrix, bounds_vector, objective_vector):
    minimum_bound = min(bounds_vector)
    
    initial_solution = np.zeros((coeff_matrix.shape[1] + 1, 1))
    initial_solution[-1] = minimum_bound
    
    current_coeff = coeff_matrix
    current_bounds = bounds_vector  
    current_objective = objective_vector
    
    if initial_solution.shape != objective_vector.shape:
        problem_rows, problem_cols = coeff_matrix.shape
        current_coeff = construct_augmented_constraint_matrix(coeff_matrix, problem_rows, problem_cols)
        current_bounds, current_objective = adjust_bounds_and_objectives(bounds_vector, problem_cols)

    return current_coeff, current_bounds, np.hstack(current_objective), np.hstack(initial_solution)

In [12]:
# Initialize tracking variables and iteration parameters
def setup_vertex_search_variables(current_solution, objective_vector):
    cost_history = []
    position_history = []
    cost_history.append(np.dot(objective_vector, current_solution))
    position_history.append(current_solution)
    current_position = current_solution
    step_counter = 0
    display_frequency = 1
    return cost_history, position_history, current_position, step_counter, display_frequency

# Display progress and adjust frequency based on iteration count
def show_search_progress(step_number, matrix_rank, display_freq):
    if step_number % display_freq == 0:
        print(f"Iteration: {step_number} - Rank: {matrix_rank}")
        
    updated_freq = display_freq
    if step_number > 300:
        updated_freq = 1000
    elif step_number > 10000:
        updated_freq = 10000
    return updated_freq

In [13]:
# Compute movement direction using null space or random vector
def calculate_movement_direction(active_constraints, inactive_constraints):
    if len(active_constraints) == 0:
        direction_vector = np.random.rand(inactive_constraints.shape[-1])
    else:
        null_space_basis = null_space(active_constraints)
        direction_vector = null_space_basis[:, 0]
    return direction_vector

# Calculate step ratios for all inactive constraints
def calculate_step_ratios(bounds_vec, constraint_mask, inactive_rows, current_pos, current_dir, tolerance):
    step_ratios = []
    for bound_val, constraint_row in zip(bounds_vec[~constraint_mask], inactive_rows):
        denominator = np.dot(constraint_row, current_dir)
        if abs(denominator) > tolerance:
            ratio = (bound_val - np.dot(constraint_row, current_pos)) / denominator
            step_ratios.append(ratio)
    return step_ratios

In [14]:
# Filter ratios to remove invalid values
def filter_valid_ratios(valid_ratios, tolerance):
    filtered_ratios = []
    for ratio in valid_ratios:
        if not(ratio == np.inf or abs(ratio) < tolerance):
            filtered_ratios.append(ratio)
    return filtered_ratios

# Calculate optimal step size with direction reversal if needed
def compute_optimal_step_size(bounds_vec, constraint_mask, inactive_rows, current_pos, movement_dir):
    direction_attempts = [movement_dir, -1 * movement_dir]
    tolerance = 1e-10
    
    for attempt_idx in range(2):
        current_dir = direction_attempts[attempt_idx]
        
        step_ratios = calculate_step_ratios(bounds_vec, constraint_mask, inactive_rows, current_pos, current_dir, tolerance)
        valid_ratios = [ratio for ratio in step_ratios if ratio > 0]
        filtered_ratios = filter_valid_ratios(valid_ratios, tolerance)
        
        if len(filtered_ratios) > 0:
            return min(filtered_ratios), current_dir
    
    return 0, movement_dir

In [15]:
# Update position and recalculate constraint classifications
def update_position_and_constraints(constraint_matrix, bounds_vector, old_pos, step_length, direction):
    new_position = old_pos + step_length * direction
    updated_mask, updated_active, updated_inactive = analyze_constraints(constraint_matrix, new_position, bounds_vector)
    return new_position, updated_mask, updated_active, updated_inactive

# Calculate matrix rank for constraint classification
def compute_constraint_rank(constraint_rows):
    return 0 if len(constraint_rows) == 0 else np.linalg.matrix_rank(constraint_rows)

In [16]:
# Execute single iteration of vertex search process
def execute_vertex_search_iteration(coeff_matrix, bounds_vector, current_pos, objective_vector, 
                                  constraint_mask, active_rows, inactive_rows, iteration_count, 
                                  current_rank, freq, cost_track, position_track):
    iteration_count += 1
    freq = show_search_progress(iteration_count, current_rank, freq)
    
    movement_direction = calculate_movement_direction(active_rows, inactive_rows)
    optimal_step, final_direction = compute_optimal_step_size(bounds_vector, constraint_mask, inactive_rows, current_pos, movement_direction)
    
    current_pos, constraint_mask, active_rows, inactive_rows = update_position_and_constraints(
        coeff_matrix, bounds_vector, current_pos, optimal_step, final_direction)
    
    current_rank = compute_constraint_rank(active_rows)
    cost_track.append(np.dot(objective_vector, current_pos))
    position_track.append(current_pos)
    
    return current_pos, constraint_mask, active_rows, inactive_rows, iteration_count, current_rank, freq

In [17]:
# Transform feasible point to vertex using iterative approach
def transform_feasible_point_to_vertex(coeff_matrix, bounds_vector, initial_point, objective_vector, problem_dimension):
    cost_track, position_track, current_pos, iteration_count, freq = setup_vertex_search_variables(initial_point, objective_vector)
    
    constraint_mask, active_rows, inactive_rows = analyze_constraints(coeff_matrix, initial_point, bounds_vector)
    current_rank = compute_constraint_rank(active_rows)
    
    if current_rank == problem_dimension:
        return current_pos, cost_track, position_track
    else:
        print("Feasible point is not a vertex. Searching for a vertex...")
    
    max_iterations = 1000
    for step_idx in range(max_iterations):
        if current_rank == problem_dimension:
            break
            
        current_pos, constraint_mask, active_rows, inactive_rows, iteration_count, current_rank, freq = execute_vertex_search_iteration(
            coeff_matrix, bounds_vector, current_pos, objective_vector, constraint_mask, 
            active_rows, inactive_rows, iteration_count, current_rank, freq, cost_track, position_track)
    
    if not(is_degenerate(active_rows)):
        return (current_pos, cost_track, position_track)
    else:
        return (None,)

In [18]:
# Initialize optimization tracking variables
def setup_optimization_tracking(initial_vertex, cost_vector):
    cost_history = []
    vertex_history = []
    current_vertex = initial_vertex
    updated_vertex = initial_vertex
    cost_history.append(np.dot(cost_vector, initial_vertex))
    vertex_history.append(initial_vertex)
    step_count = 0
    display_interval = 1
    return cost_history, vertex_history, current_vertex, updated_vertex, step_count, display_interval

# Extract profitable movement directions from feasible directions
def extract_profitable_directions(direction_matrix, objective_coeffs):
    beneficial_directions = []
    for direction_vec in direction_matrix:
        if np.dot(direction_vec, objective_coeffs) > 0:
            beneficial_directions.append(direction_vec)
    return beneficial_directions

In [19]:
# Calculate step sizes for vertex movement
def calculate_vertex_step_sizes(bounds_vec, active_mask, inactive_constraints, current_vertex, movement_direction):
    step_candidates = []
    for bound_val, constraint_vec in zip(bounds_vec[~active_mask], inactive_constraints):
        denominator = np.dot(constraint_vec, movement_direction)
        if denominator != 0:
            step_ratio = (bound_val - np.dot(constraint_vec, current_vertex)) / denominator
            step_candidates.append(step_ratio)
    
    positive_steps = [step for step in step_candidates if step > 0]
    return positive_steps

# Check for unbounded optimization problem
def check_unbounded_condition(valid_steps, tolerance=1e-10):
    if len(valid_steps) == 0:
        return True, None
    
    filtered_steps = []
    for step_val in valid_steps:
        if not(step_val == np.inf or abs(step_val) < tolerance):
            filtered_steps.append(step_val)
    
    if len(filtered_steps) == 0:
        return True, None
    
    return False, min(filtered_steps)

In [20]:
# Process optimization direction and step calculation
def process_optimization_direction(active_constraints, bounds_vec, active_mask, inactive_constraints, current_vertex, cost_vec, step_count):
    if is_degenerate(active_constraints):
        return None, None, None, None, step_count, True
    
    direction_matrix = compute_movement_vectors(active_constraints).T
    beneficial_directions = extract_profitable_directions(direction_matrix, cost_vec)
    
    # If no beneficial directions, we've reached optimal - terminate successfully
    if not beneficial_directions:
        return current_vertex, None, None, False, step_count, True
    
    # Prevent infinite loops
    if step_count > 10:
        return current_vertex, None, None, False, step_count, True
    
    selected_direction = beneficial_directions[0]
    step_candidates = calculate_vertex_step_sizes(bounds_vec, active_mask, inactive_constraints, current_vertex, selected_direction)
    
    is_unbounded, optimal_step = check_unbounded_condition(step_candidates)
    
    return selected_direction, step_candidates, optimal_step, is_unbounded, step_count, False

In [21]:
# Execute single vertex-to-vertex optimization step
def execute_optimization_step(constraint_matrix, bounds_vec, current_vertex, cost_vec, step_count, display_freq):
    step_count += 1
    if step_count % display_freq == 0:
        print(f"Iteration: {step_count}")
    
    active_mask, active_constraints, inactive_constraints = analyze_constraints(constraint_matrix, current_vertex, bounds_vec)
    
    selected_direction, step_candidates, optimal_step, is_unbounded, step_count, early_termination = process_optimization_direction(
        active_constraints, bounds_vec, active_mask, inactive_constraints, current_vertex, cost_vec, step_count)
    
    if early_termination:
        if selected_direction is None:
            # Degeneracy case
            return None, None, None, None, step_count, True
        else:
            # Optimal solution found (no beneficial directions)
            return selected_direction, None, None, None, step_count, False
    
    if is_unbounded or optimal_step is None:
        return None, None, None, None, step_count, True
    
    new_vertex = current_vertex + optimal_step * selected_direction
    return new_vertex, new_vertex, optimal_step, selected_direction, step_count, False

In [22]:
# Optimize from vertex to optimal vertex
def optimize_vertex_to_vertex(coeff_matrix, bounds_vector, initial_vertex, objective_vector, problem_dimensions):
    cost_track, vertex_track, current_pos, updated_pos, iteration_num, freq = setup_optimization_tracking(initial_vertex, objective_vector)
    
    max_iterations = 1000
    for step_idx in range(max_iterations):
        current_pos, updated_pos, step_size, direction_vec, iteration_num, should_terminate = execute_optimization_step(
            coeff_matrix, bounds_vector, current_pos, objective_vector, iteration_num, freq)
        
        if should_terminate:
            if current_pos is None:
                return None, cost_track, vertex_track
            else:
                return (None,)
        
        if current_pos is not None and updated_pos is None:
            return current_pos, cost_track, vertex_track
        
        if updated_pos is not None:
            cost_track.append(np.dot(objective_vector, updated_pos))
            vertex_track.append(updated_pos)
    
    return current_pos, cost_track, vertex_track

In [23]:
# Check if origin is feasible and setup auxiliary problem if needed
def check_origin_feasibility(constraint_matrix, bounds_vector, objective_coeffs):
    origin_is_feasible = False
    if np.all(bounds_vector >= 0):
        origin_is_feasible = True
        initial_solution = np.zeros(objective_coeffs.shape)
        return (initial_solution,), origin_is_feasible
    else:
        augmented_matrix, modified_bounds, auxiliary_obj, auxiliary_solution = create_phase_one_formulation(constraint_matrix, bounds_vector, objective_coeffs)
        problem_rows, problem_cols = len(modified_bounds), len(auxiliary_obj)
        return (augmented_matrix, modified_bounds, auxiliary_obj, auxiliary_solution, problem_rows, problem_cols), origin_is_feasible

# Initialize optimization parameters for auxiliary problem
def initialize_auxiliary_optimization(augmented_data):
    constraint_matrix, bounds_original, auxiliary_obj, auxiliary_solution, problem_rows, problem_cols = augmented_data
    perturbation_epsilon = 0.1
    retry_count = 0
    current_bounds = bounds_original
    return constraint_matrix, bounds_original, auxiliary_solution, auxiliary_obj, problem_cols, perturbation_epsilon, retry_count, current_bounds

In [24]:
# Execute single optimization attempt with degeneracy handling
def execute_optimization_attempt(coeff_matrix, bounds_vec, initial_vertex, objective_vec, dimensions, attempt_num, epsilon_val, original_bounds):
    if attempt_num > 0:
        bounds_vec, epsilon_val = remove_degeneracy_condition(original_bounds, epsilon_val)
    
    phase1_result = transform_feasible_point_to_vertex(coeff_matrix, bounds_vec, initial_vertex, objective_vec, dimensions)
    if len(phase1_result) == 1:
        return None, attempt_num + 1, epsilon_val, bounds_vec
    
    vertex_position, cost_history, position_history = phase1_result
    
    phase2_result = optimize_vertex_to_vertex(coeff_matrix, bounds_vec, vertex_position, objective_vec, dimensions)
    if len(phase2_result) == 1:
        return None, attempt_num + 1, epsilon_val, bounds_vec
    
    optimal_vertex, optimization_costs, optimization_path = phase2_result
    return (vertex_position, cost_history, position_history, optimal_vertex, optimization_costs, optimization_path), attempt_num, epsilon_val, bounds_vec

In [25]:
# Process optimization results and determine solution status
def process_optimization_results(optimal_solution, original_dimension):
    if optimal_solution is None:
        print("The problem is unbounded!")
        return None, None, None, None, None, None
    
    vertex_pos, cost_track, path_track, final_vertex, final_costs, final_path = optimal_solution
    
    if np.all(final_vertex == None):
        solution_status = None
    else:
        solution_status = -1
    
    if solution_status is None:
        return None, vertex_pos, cost_track, path_track, final_costs, final_path
    else:
        feasible_point = final_vertex[:original_dimension]
        return feasible_point, vertex_pos, cost_track, path_track, final_costs, final_path
    
# Handle final result processing and return values
def handle_final_results(solution_data, original_bounds_feasible):
    if original_bounds_feasible:
        return solution_data
    
    feasible_pt, vertex_pt, cost_hist, path_hist, opt_costs, opt_path = solution_data
    
    if feasible_pt is None:
        print("The modified LP is unbounded! Therefore the given problem is unbounded!")
        return (None, None, vertex_pt, cost_hist, path_hist, None, opt_costs, opt_path)
    else:
        return (feasible_pt, -1, vertex_pt, cost_hist, path_hist, feasible_pt, opt_costs, opt_path)

In [26]:
# Find initial feasible solution using phase-one method
def locate_initial_feasible_solution(constraint_matrix, bounds_vector, objective_coeffs):
    initial_data, origin_feasible = check_origin_feasibility(constraint_matrix, bounds_vector, objective_coeffs)
    
    if origin_feasible:
        return initial_data
    
    # initialize_auxiliary_optimization returns: (constraint_matrix, bounds_original, auxiliary_solution, auxiliary_obj, problem_cols, perturbation_epsilon, retry_count, current_bounds)
    coeff_mat, bounds_orig, start_point, cost_vec, dim_count, epsilon, attempts, current_bounds = initialize_auxiliary_optimization(initial_data)
    
    max_attempts = 100
    for attempt_idx in range(max_attempts):
        result_data, attempts, epsilon, current_bounds = execute_optimization_attempt(
            coeff_mat, current_bounds, start_point, cost_vec, dim_count, attempts, epsilon, bounds_orig)
        
        if result_data is not None:
            break
    
    processed_results = process_optimization_results(result_data, len(objective_coeffs))
    final_results = handle_final_results(processed_results, origin_feasible)
    
    return final_results

In [27]:
constraint_matrix, start_point, constraint_bounds, objective_coeffs = parse_csv_input('testcase_2.csv', has_initial_point=False)
num_constraints, num_variables = len(constraint_bounds), len(objective_coeffs)

In [28]:
print("Objective coefficients:", objective_coeffs)
print("Constraint bounds:", constraint_bounds)
print("Constraint matrix:")
print(constraint_matrix)
print(f"Problem dimensions: {num_constraints} constraints, {num_variables} variables")

Objective coefficients: [0. 1.]
Constraint bounds: [-1. 25. -9.  1. 15.]
Constraint matrix:
[[ 0. -1.]
 [ 2.  1.]
 [-2. -1.]
 [-2.  1.]
 [ 2. -1.]]
Problem dimensions: 5 constraints, 2 variables


In [29]:
# Process optimization results and extract feasible solution
def process_feasible_solution_results(optimization_outputs, objective_dimension):
    if len(optimization_outputs) > 1:
        optimal_solution, solution_status, vertex_point, phase1_costs, phase1_path, final_solution, phase2_costs, phase2_path = optimization_outputs
        if solution_status != None:
            feasible_point = optimal_solution[:objective_dimension]
        else:
            feasible_point = None
        return feasible_point, solution_status, vertex_point, phase1_costs, phase1_path, final_solution, phase2_costs, phase2_path
    else:
        feasible_point = optimization_outputs[0]
        solution_status = -1
        return feasible_point, solution_status, None, None, None, None, None, None

outputs = locate_initial_feasible_solution(constraint_matrix, constraint_bounds, objective_coeffs)
z, modified_z_optimal, z_new, feas2vert_z_all_cost, feas2vert_z_all, z_optimal, vert2vert_z_all_cost, vert2vert_z_all = process_feasible_solution_results(outputs, len(objective_coeffs))

Feasible point is not a vertex. Searching for a vertex...
Iteration: 1 - Rank: 2
Iteration: 1
Iteration: 2
Iteration: 3


In [30]:
# Transform feasible point to vertex with degeneracy handling
def execute_phase1_transformation(constraint_matrix, bounds_vector, initial_solution, objective_vector, problem_dimensions, retry_attempt, perturbation_epsilon):
    if retry_attempt > 0:
        current_bounds, perturbation_epsilon = remove_degeneracy_condition(bounds_vector, perturbation_epsilon)
    else:
        current_bounds = bounds_vector
    
    phase1_results = transform_feasible_point_to_vertex(constraint_matrix, current_bounds, initial_solution, objective_vector, problem_dimensions)
    if len(phase1_results) == 1:
        retry_attempt += 1
        return None, retry_attempt, perturbation_epsilon, current_bounds
    
    initial_vertex, feasible_costs, feasible_path = phase1_results
    return (initial_vertex, feasible_costs, feasible_path), retry_attempt, perturbation_epsilon, current_bounds

# Optimize from vertex to optimal vertex with result processing
def execute_phase2_optimization(constraint_matrix, current_bounds, initial_vertex, objective_vector, problem_dimensions, feasible_costs, feasible_path, retry_attempt):
    phase2_results = optimize_vertex_to_vertex(constraint_matrix, current_bounds, initial_vertex, objective_vector, problem_dimensions)
    if len(phase2_results) == 1:
        retry_attempt += 1
        return None, retry_attempt
    
    optimal_solution, optimization_costs, optimization_path = phase2_results
    
    if optimal_solution is None:
        return (None, initial_vertex, feasible_costs, feasible_path, None, None), retry_attempt
    else:
        return (optimal_solution, initial_vertex, feasible_costs, feasible_path, optimization_costs, optimization_path), retry_attempt


In [31]:
# Execute optimization with degeneracy handling using decomposed functions
def execute_optimization_with_degeneracy_handling(constraint_matrix, bounds_vector, initial_solution, objective_vector, problem_dimensions):
    perturbation_epsilon = 0.1
    retry_attempt = 0
    max_attempts = 10
    
    for attempt_idx in range(max_attempts):
        phase1_result, retry_attempt, perturbation_epsilon, current_bounds = execute_phase1_transformation(
            constraint_matrix, bounds_vector, initial_solution, objective_vector, problem_dimensions, retry_attempt, perturbation_epsilon)
        
        if phase1_result is None:
            continue
        
        initial_vertex, feasible_costs, feasible_path = phase1_result
        
        final_result, retry_attempt = execute_phase2_optimization(
            constraint_matrix, current_bounds, initial_vertex, objective_vector, problem_dimensions, feasible_costs, feasible_path, retry_attempt)
        
        if final_result is not None:
            return final_result
    
    return None, None, None, None, None, None

In [32]:

# Handle unbounded case or execute optimization
def process_optimization_execution(solution_status, constraint_matrix, bounds_vector, feasible_point, objective_vector, problem_dimensions):
    if solution_status == None:
        return None, None, None, None, None, None
    else:
        return execute_optimization_with_degeneracy_handling(constraint_matrix, bounds_vector, feasible_point, objective_vector, problem_dimensions)

# Execute main optimization process
final_solution, vertex_point, phase1_costs, phase1_path, phase2_costs, phase2_path = process_optimization_execution(
    modified_z_optimal, constraint_matrix, constraint_bounds, z, objective_coeffs, num_variables)

Feasible point is not a vertex. Searching for a vertex...
Iteration: 1 - Rank: 0
Iteration: 2 - Rank: 1
Iteration: 1


In [33]:
# Display simplified optimization results
def display_optimization_results(solution_status, final_solution, vertex_point, phase1_costs, phase1_path, phase2_costs, phase2_path):
    if solution_status == None:
        print("Problem is unbounded")
        return
    
    if phase1_path is not None:
        print("Feasible to vertex sequence:")
        for i, vertex in enumerate(phase1_path):
            print(f"  Step {i+1}: [{', '.join(f'{x:.3f}' for x in vertex)}] (cost: {phase1_costs[i]:.3f})")
    
    if phase2_path is not None:
        print("\nVertex to optimal sequence:")
        for i, vertex in enumerate(phase2_path):
            print(f"  Step {i+1}: [{', '.join(f'{x:.3f}' for x in vertex)}] (cost: {phase2_costs[i]:.3f})")
        
        if final_solution is None:
            print("Result: Problem is unbounded")
        else:
            print(f"\nOptimal solution: [{', '.join(f'{x:.3f}' for x in final_solution)}]")
    else:
        print("Optimization incomplete due to degeneracy")

In [34]:
# Generate simple summary
def generate_optimization_summary(solution_status, final_solution, vertex_point, phase1_costs, phase1_path, phase2_costs, phase2_path):
    if solution_status == None:
        print("\nStatus: Unbounded problem")
    elif final_solution is None:
        print("\nStatus: Unbounded during optimization")
    else:
        print("\nStatus: Optimal solution found")
        print(f"Solution: [{', '.join(f'{x:.6f}' for x in final_solution)}]")
        if phase2_costs:
            print(f"Objective value: {phase2_costs[-1]:.6f}")

# Display results and summary
display_optimization_results(modified_z_optimal, final_solution, vertex_point, phase1_costs, phase1_path, phase2_costs, phase2_path)
generate_optimization_summary(modified_z_optimal, final_solution, vertex_point, phase1_costs, phase1_path, phase2_costs, phase2_path)

Feasible to vertex sequence:
  Step 1: [6.000, 7.000] (cost: 7.000)
  Step 2: [7.816, 9.367] (cost: 9.367)
  Step 3: [6.000, 13.000] (cost: 13.000)

Vertex to optimal sequence:
  Step 1: [6.000, 13.000] (cost: 13.000)

Optimal solution: [6.000, 13.000]

Status: Optimal solution found
Solution: [6.000000, 13.000000]
Objective value: 13.000000
